# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Знакомство с данными

In [163]:
import pandas as pd
from pymystem3 import Mystem

In [164]:
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [165]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Типы данных в столбцах: float64 - 2, int64 - 5, object - 5.

Согласно документации к данным: 
* `children` — количество детей в семье
* `days_employed` — трудовой стаж в днях
* `dob_days` — возраст клиента в годах
* `education` — образование клиента
* `education_id` — идентификатор образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — доход в месяц
* `purpose` — цель получения кредита

В названиях колонок стилистических и иных нарушений нет.

Количество значений в столбцах различается, значит в данных есть пропущенные значения. 

**Вывод**

Каждая строка таблицы описывает клиента банка, начиная с количества детей и заканчивая его доходом в месяц. В 2 колонках описывается взаимодействие клиента с банком: имел ли задолженность по возврату кредитов и цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, которые нужно устранить, чтобы двигаться дальше.

## Предобработка данных

### Обработка пропусков

In [166]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в доходах заполним медианным значением дохода исходя из типа занятости и уровня дохода.

In [167]:
data_grouped = data.pivot_table(index='income_type', columns='education_id', values='total_income', aggfunc='median')
display(data_grouped)

education_id,0,1,2,3,4
income_type,,,,,
безработный,202722.511368,59956.991984,NaN,NaN,NaN
в декрете,NaN,53829.130729,NaN,NaN,NaN
госслужащий,172511.107016,136652.970357,160592.345303,148339.290825,111392.231107
компаньон,201785.400018,159070.690289,179867.152890,136798.905143,NaN
пенсионер,144240.768611,114842.854099,120136.896353,102598.653164,177088.845999
предприниматель,499163.144947,NaN,NaN,NaN,NaN
сотрудник,165640.744634,136555.108821,151308.937846,125994.910603,198570.757322
студент,98201.625314,NaN,NaN,NaN,NaN


In [168]:
def func(row):
    if pd.isna(row['total_income']):
        return data_grouped.loc[row['income_type'], row['education_id']]
    return row['total_income']   

data['total_income'] = data.apply(func, axis=1)

Данные в колонке `days_employed` не влияют на наше исследование и пропуски там можно заменить нулём. 

In [169]:
data['days_employed'] = data['days_employed'].fillna(0)

In [170]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски обнаружены в столбцах `days_employed` и `total_income`. Что характерно количество пропусков одинаково и там и там. Вероятно их появление взаимосвязано. 

Возможные причины: 
* получение кредита по одному документу(например, паспорт), без документов подтверждающих стаж и доход
* ошибка внесения данных оператором
* программная ошибка при передаче внесённых данных
* ошибка работы сайта, если заявка заполнялась онлайн
* ошибка при сохранении/чтении документа

Пропущенные значения в столбце `days_employed` не влияют на текущее исследование, но данные в других столбцах представляют для нас ценность, поэтому эти строки удалять не будем, а заменим там значения на `0`.

Данные в столбце `total_income` нам необходимы, чтобы установить взаимосвязь между уровнем дохода и возвратом кредита в срок. Пропущенные значения в столбце `total_income` составляют ~10% - это большое количество, которое может сильно повлиять на итоговый вывод, поэтому их также не удаляем, а заменим на характерное значение.  Восстановить точные данные в данном случае нет возможности, поэтому использовалось медианное значение для заполнения, так как присутствуют выделяющиеся значения.   

### Замена типа данных

In [171]:
data['total_income'] = data['total_income'].astype('int')
data['debt'] = data['debt'].astype('str')

**Вывод**

Для приведения значениий в столбце `total_income` к целочисленному формату был использован метод `astype()`.

### Обработка дубликатов

In [172]:
data.duplicated().sum()

54

In [173]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [174]:
data['education'] = data['education'].str.lower()

In [175]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [176]:
data.duplicated().sum()

71

In [177]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

При первоначальной проверке было обнаружено 54 явных дубликата. Но в столбце `education` одни и те же значения написаны по разному. Привели их к нижнему регистру для единообразия. После этого было обнаружено уже 71 дубликат. 

Причины возникновения дубликатов могут быть как человеческий фактор, так и ошибки в работе систем.

### Лемматизация

In [178]:
m = Mystem()
data['purpose'] = data['purpose'].apply(m.lemmatize)
data['purpose'].head()

0                 [покупка,  , жилье, \n]
1       [приобретение,  , автомобиль, \n]
2                 [покупка,  , жилье, \n]
3    [дополнительный,  , образование, \n]
4               [сыграть,  , свадьба, \n]
Name: purpose, dtype: object

**Вывод**

Полученные леммы в дальнейшем послужат для категоризации целей кредитов. 

### Категоризация данных

**Категоризация данных по количеству детей**

In [179]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

При рассмотрении данных столбца `children` обнаружены нехарактерные значения `"-1"` и `"20"`. Можно предположить, что это опечатки при вводе данных (соответственно `1` и `2`). Но если `2` на раскладке клавиатуры находится рядом с `0`, то `-` и `1`, как правило, не соседствуют между собой. С большой долей вероятности утверждать, что это опечатка, сложно. Поэтому в исследовании эти значения учитываться не будут, тем более они составляют только 0,57% от общего количества.

In [180]:
data_filtered = data[data['children'] >= 0]
data_filtered = data_filtered[data_filtered['children'] < 20]
data_filtered['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [181]:
def family_group(children):
    if children == 0:
        return 'нет детей'
    if children < 3:
        return 'есть дети'
    return 'многодетные'

data_filtered['family_group'] = data_filtered['children'].apply(family_group)
data_filtered.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,family_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]",есть дети
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",есть дети
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]",нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]",многодетные
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]",нет детей


**Категоризация данных по уровню дохода**

In [182]:
data['total_income_group'] = pd.qcut(data['total_income'], q=5, labels=['низкий', 'ниже среднего', 'средний', 'выше среднего', 'высокий'])
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]",высокий
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",ниже среднего
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]",средний
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]",высокий
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]",средний


Всех клиентов с помощью метода `pd.qcut()` разделили на пять равных по количеству групп и присвоили им категорию дохода. 

**Категоризация по цели кредита**

In [183]:
def purpose_category(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    return 'образование'

data['purpose_category'] = data['purpose'].apply(purpose_category)
data['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

**Вывод**

Провели объединение данных в группы, что в дальнейшем поможет их проанализировать и дать ответы на вопросы Заказчика. 

## Исследовательский анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [184]:
data_filtered_pivot = data_filtered.pivot_table(index='family_group', columns='debt', values='children', aggfunc='count')
display(data_filtered_pivot)

debt,0,1
family_group,,
есть дети,6222,638
многодетные,349,31
нет детей,13028,1063


Где `0` - нет задолженности, `1` - есть задолженность.

Самое большое количество должников в категории `нет детей`, но там и самое большое количество заёмщиков. Посчитаем долю должников от общего количества в каждой категории и сравним их.

In [185]:
data_filtered_pivot['ratio'] = data_filtered_pivot['1']/(data_filtered_pivot['1'] + data_filtered_pivot['0'])
display(data_filtered_pivot.sort_values(by='ratio'))

debt,0,1,ratio
family_group,,,
нет детей,13028,1063,0.075438
многодетные,349,31,0.081579
есть дети,6222,638,0.093003


**Вывод**

Самая низкая доля должников в группе `нет детей`. Самая высокая где 1-2 ребенка. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [186]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [187]:
data_family_status_pivot = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
display(data_family_status_pivot)

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


In [188]:
data_family_status_pivot['ratio'] = data_family_status_pivot['1']/(data_family_status_pivot['1'] + data_family_status_pivot['0'])
data_family_status_pivot.sort_values(by='ratio')

debt,0,1,ratio
family_status,,,
вдовец / вдова,896,63,0.065693
в разводе,1110,85,0.071130
женат / замужем,11408,931,0.075452
гражданский брак,3763,388,0.093471
Не женат / не замужем,2536,274,0.097509


**Вывод**

Самая низкая доля должников в категории `вдовец/вдова`, затем рядом `в разводе` и клиенты в браке. Самые большие доли в категориях `гражданский брак` и `не женат/не замужем`.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [189]:
data_income_pivot = data.pivot_table(index='total_income_group', columns='debt', values='total_income', aggfunc='count')
display(data_income_pivot)

debt,0,1
total_income_group,,
низкий,3947,344
ниже среднего,3932,359
средний,3904,386
выше среднего,3939,352
высокий,3991,300


In [190]:
data_income_pivot['ratio'] = data_income_pivot['1']/(data_income_pivot['1']+data_income_pivot['0'])
display(data_income_pivot.sort_values(by='ratio'))

debt,0,1,ratio
total_income_group,,,
высокий,3991,300,0.069914
низкий,3947,344,0.080168
выше среднего,3939,352,0.082032
ниже среднего,3932,359,0.083663
средний,3904,386,0.089977


**Вывод**

Самая низкая доля должников у клиентов с высоким доходом, самая высокая - у клиентов со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [191]:
data_purpose_pivot = data.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
display(data_purpose_pivot)

debt,0,1
purpose_category,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [192]:
data_purpose_pivot['ratio'] = data_purpose_pivot['1']/(data_purpose_pivot['1']+data_purpose_pivot['0'])
display(data_purpose_pivot.sort_values(by='ratio'))

debt,0,1,ratio
purpose_category,,,
недвижимость,10029,782,0.072334
свадьба,2138,186,0.080034
образование,3643,370,0.092200
автомобиль,3903,403,0.093590


**Вывод**

Самые низкие доли задолженностей в категориях `недвижимость` и `свадьба`. `Образование` и `автомобиль` напротив имеют самые высокие доли.

## Общий вывод

Мы проверили четыре гипотезы и установили:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

Зависимость прослеживается. Самая низкая доля должников в группе `нет детей` - 7.54%. Самая высокая где 1-2 ребенка - 9.3%.

2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

Видна четкая разница между клиентами в браке, либо бывшими в браке и холостыми, либо не оформившими отношения официально не в пользу последних. 

Самая низкая доля - вдовец/вдова - 6,57%; самая высокая - 9,75%.

3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Зависимость прослеживается. Самая низкая доля - клиенты с высоким доходом - 6,99%; самая высокая доля - со средним доходом - 8,99%. Что интересно, клиенты с низким доходом находятся на втором месте по надёжности.

4. Как разные цели кредита влияют на его возврат в срок?

Также видна взаимосвязь между целью кредита и его возвратом в срок. Те, кто берут кредит на операции с недвижимостью допускают меньше просрочек оплаты - 7,23%. Больше всего должников в категории клиентов, берущих кредит на покупку автомобиля - 9,35%. 

В общем и целом, доли должников во всех категориях не превышают 10%. Во всех категориях прослеживаются градации групп клиентов, которые наиболее ответственно подходят к выплате кредита и не очень. По всем четырём вопросам можем ответить, что зависимость возврата кредита в срок присутствует. 
